In [1]:
# import required python modules
import pandas as pd
import os.path as os_path
import csv
import os

In [2]:
# set global variables for start and end year -- these must be data type str
START_YEAR = '2015'
END_YEAR = '2022'

In [3]:
import numpy
print(numpy.__version__)
print(pd.__version__)
import sys
print(sys.version)

1.24.3
1.5.1
3.8.10 (tags/v3.8.10:3d8993a, May  3 2021, 11:48:03) [MSC v.1928 64 bit (AMD64)]


In [4]:
# Get the current working directory
cwd = os.getcwd()

# Print the current working directory
print("Current working directory:", cwd)


Current working directory: C:\Users\user\Downloads


In [5]:
DATA_DIR = '..\data'
OUTPUT_DIR = '..\output'
PATEX_DIR = 'C:\\Users\\user\\Downloads\\BulkData2023'
PATENTSVIEW_DIR = "C:\\Users\\user\\Downloads\\data_patentsview_20231231"
OUTPUT_FNAME = 'nj_univ_appl_inv_gender.csv'

In [6]:
def id_rutgers_stevens_njit(df, text_col, output_col_suffix=''):
    '''Create columns to identify if text in 'text_col' is associated with Rutgers, Stevens, or NJIT

    This function is tailored for Rutgers University, Stevens Institute of Technology, and
    New Jersey Institute of Technology. Use a set of keywords (whole words or simplified) to
    identify if the text in 'text_col' contains the keywords assocated with each target
    university. A binary output column, one for each target university, identifies the result
    as True or False; to increase flexibility, add a suffix to the output columns.

    Rutgers = rutgers AND (univer OR jersey)
    Stevens = stevens AND institute
    NJIT = jersey AND institute AND technol

    :param df (dataframe): dataframe to analyze
    :param text_col (str): column containing text w/names
    :param output_col_suffix (str): suffix to add to end of output columns (default='')
    :return df (dataframe): dataframe with binary output columns indicating university
    '''
    # lowercase text in text_col
    df[text_col] = df[text_col].str.lower()
    # analyze Rutgers
    df['rutgers_'+output_col_suffix] = df[text_col].apply(lambda x: \
                      ('rutgers' in x) and (('univer' in x) or ('jersey' in x)))
    # analyze Stevens
    df['stevens_'+output_col_suffix] = df[text_col].apply(lambda x: \
                      ('stevens' in x) and ('institute' in x) and ('tech' in x))
    # analyze NJIT
    df['njit_'+output_col_suffix] = df[text_col].apply(lambda x: \
                      ('jersey' in x) and ('institute' in x) and ('technol' in x))

    #analyse princeton
    df['princeton_'+output_col_suffix] = df[text_col].apply(lambda x: ('princeton' in x) and ('univer' in x))

    #analyse rowan
    df['rowan_'+output_col_suffix] = df[text_col].apply(lambda x: ('rowan' in x) and ('univer' in x))

    #analyse NJCU
    df['NJCU_'+output_col_suffix] = df[text_col].apply(lambda x: ('jersey' in x) and ('city' in x) and ('univer' in x))

    #analyse seton
    df['seton_'+output_col_suffix] = df[text_col].apply(lambda x: ('seton' in x) and ('hall' in x))

    #analyse saint peter's
    df['saintPeters_'+output_col_suffix] = df[text_col].apply(lambda x: ('saint') and ('peter' in x) and ('univer' in x))
    
    # return updated dataframe
    return df

In [7]:
def add_appl_owner_to_dict(appl_dict, appl_id, owner):
    '''Add the owner of an application to a dictionary.
    
    The dictionary key is an application number, and values are a set of owners for that application. 
    The use of sets precludes duplicates. Use data type to ignore blank values (e.g., NaN for an owner). 
    Note: we do not use a defaultdict to simplify the later conversion to a pandas dataframe.
    :param appl_dict (dict of sets): initial application owner dictionary 
    :param appl_id (str): application number
    :param owner (str): either an individual applicant or assignee of an application
    :return appl_dict (dict of sets): updated applicaton owner dictionary    
    '''
    # check if input application id is a string
    if type(appl_id)==str:
        # if application id is not in dictionary keys then add it to the dict w/empty set
        if appl_id not in appl_dict.keys(): appl_dict[appl_id] = set()
        # if input owner is a string, add to set for that application 
        if type(owner)==str: appl_dict[appl_id].add(owner)
    return appl_dict

In [8]:
# set filepath to PatEx applicant data csv file
filepath = os_path.join(PATEX_DIR, 'all_applicants.csv')
print(filepath)

C:\Users\user\Downloads\BulkData2023\all_applicants.csv


In [9]:
with open(filepath, 'r') as file:
    for _ in range(2):
        print(file.readline(), end='')

application_number,applicant_organization,applicant_name_first,applicant_name_middle,applicant_name_last,applicant_city_name,applicant_region_code,applicant_country_code
10470614,,Societe Franco-Belge de Fabrication de,,Combustile - FBFC,Courbevoie,,FR


In [10]:
applicant_df = pd.read_csv(filepath,
                           header=0,
                           dtype=str,
                           keep_default_na=False)

In [11]:
# describe the dataframe (using transpose() put the output in a more readable format)
applicant_df.describe().transpose()

,count,unique,top,freq
application_number,6752288,5171923,PCT/US07/22376,80
applicant_organization,6752288,516808,,1478181
applicant_name_first,6752288,83274,,5318970
applicant_name_middle,6752288,25341,,6032893
applicant_name_last,6752288,208738,,5274212
applicant_city_name,6752288,131561,Tokyo,295517
applicant_region_code,6752288,267,,2610661
applicant_country_code,6752288,255,US,3060352


In [12]:
# filter data to keep only US and NJ applicants
print('Number of rows before US NJ filter: {:,}'.format(len(applicant_df)))

applicant_df = applicant_df.loc[(applicant_df['applicant_country_code']=='US') & \
                                (applicant_df['applicant_region_code']=='NJ')]

print('Number of rows after US NJ filter: {:,}'.format(len(applicant_df)))

Number of rows before US NJ filter: 6,752,288
Number of rows after US NJ filter: 122,653


In [13]:
# create column for application series code
applicant_df['series'] = applicant_df['application_number'].apply(lambda x: x[0:2])

In [14]:
# filter data to remove PCT applications -- begins with "PCT" and hence the series from above = "PC"
print('Number of rows before PCT filter: {:,}'.format(len(applicant_df)))

applicant_df = applicant_df.loc[applicant_df['series']!='PC']

print('Number of rows after PCT filter: {:,}'.format(len(applicant_df)))

Number of rows before PCT filter: 122,653
Number of rows after PCT filter: 34,269


In [15]:
# create combined applicant string - each column separated by a space
# (note: using the python .join() function would be more complicated due to our using dataframes;
#        hence, use a simple concatenation)
applicant_df['applicant_combined'] = applicant_df['applicant_organization'] + ' ' + \
                                     applicant_df['applicant_name_first'] + ' ' + \
                                     applicant_df['applicant_name_middle'] + ' ' + \
                                     applicant_df['applicant_name_last']

In [16]:
# identify applications having Rutgers, Stevens, or NJIT
applicant_df = id_rutgers_stevens_njit(applicant_df, 'applicant_combined', output_col_suffix='app')

In [17]:
# summarize results
print('Number of Rutgers applications: {:,}'.format(len(applicant_df.loc[applicant_df['rutgers_app']==True])))
print('Number of Stevens applications: {:,}'.format(len(applicant_df.loc[applicant_df['stevens_app']==True])))
print('Number of NJIT applications: {:,}'.format(len(applicant_df.loc[applicant_df['njit_app']==True])))
print('Number of princeton applications: {:,}'.format(len(applicant_df.loc[applicant_df['princeton_app']==True])))
print('Number of rowan applications: {:,}'.format(len(applicant_df.loc[applicant_df['rowan_app']==True])))
print('Number of NJCU applications: {:,}'.format(len(applicant_df.loc[applicant_df['NJCU_app']==True])))
print('Number of seton applications: {:,}'.format(len(applicant_df.loc[applicant_df['seton_app']==True])))
print('Number of saintPeters applications: {:,}'.format(len(applicant_df.loc[applicant_df['saintPeters_app']==True])))

Number of Rutgers applications: 612
Number of Stevens applications: 96
Number of NJIT applications: 103
Number of princeton applications: 422
Number of rowan applications: 69
Number of NJCU applications: 0
Number of seton applications: 7
Number of saintPeters applications: 0


In [18]:
# summarize the applicant names associated with the target universities
for univ in ['rutgers_app', 'stevens_app', 'njit_app','princeton_app','rowan_app','NJCU_app','seton_app','saintPeters_app']:
    print('Applicant names for {}'.format(univ))
    print('----------------------')
    df = applicant_df.loc[applicant_df[univ]==True]
    print(df['applicant_combined'].value_counts())
    print()

Applicant names for rutgers_app
----------------------
rutgers, the state university of new jersey              585
   rutgers, the state university of new jersey             4
rutgers, the state university                              4
rutgers university                                         4
rutgers,the state university of new jersey                 3
 rutgers, the state university of  new jersey              2
rutgers the state university of new jersey                 2
 rutgers, the state  university of new jersey              1
  rutgers, the state university of new jersey              1
rutgers, the state of university of new jersey             1
rutgers, the state univeristy of new jersey                1
rutgers, the state university of new jersy                 1
rutgers, the state of new jersey                           1
rutgers , the state university of new jersey               1
rutgers, the state university of new jersey; offic         1
Name: applicant_combined, dtyp

In [19]:
## keep only observations w/target universities
print('Number of rows before removing non-target universities: {:,}'.format(len(applicant_df)))

applicant_df = applicant_df.loc[(applicant_df['rutgers_app']==True) | \
                                (applicant_df['stevens_app']==True) | \
                                (applicant_df['njit_app']==True) | \
                                (applicant_df['princeton_app']==True) | \
                                (applicant_df['rowan_app']==True) | \
                                (applicant_df['NJCU_app']==True) | \
                                (applicant_df['seton_app']==True) | \
                                (applicant_df['saintPeters_app']==True)]

print('Number of rows after removing non-target universities: {:,}'.format(len(applicant_df)))

Number of rows before removing non-target universities: 34,269
Number of rows after removing non-target universities: 1,309


In [20]:
# keep only the columns we need
applicant_df = applicant_df[['application_number','rutgers_app', 'stevens_app', \
                             'njit_app','princeton_app','rowan_app','NJCU_app','seton_app','saintPeters_app']]

In [21]:
# create a consoldiated university variable
applicant_df.loc[applicant_df['rutgers_app']==True,'univ'] = 'Rutgers'
applicant_df.loc[applicant_df['stevens_app']==True,'univ'] = 'Stevens'
applicant_df.loc[applicant_df['njit_app']==True,'univ'] = 'NJIT'
applicant_df.loc[applicant_df['princeton_app']==True,'univ'] = 'Princeton'
applicant_df.loc[applicant_df['rowan_app']==True,'univ'] = 'Rowan'
applicant_df.loc[applicant_df['NJCU_app']==True,'univ'] = 'NJCU'
applicant_df.loc[applicant_df['seton_app']==True,'univ'] = 'Seton'
applicant_df.loc[applicant_df['saintPeters_app']==True,'univ'] = 'saintPeters'
# print results
print(applicant_df['univ'].value_counts())

Rutgers      612
Princeton    422
NJIT         103
Stevens       96
Rowan         69
Seton          7
Name: univ, dtype: int64


In [22]:
# define columns we want to use (see above)
'''
col_g_assignee  = ['patent_id','disambig_assignee_organization',
                   'disambig_assignee_individual_name_first', 'disambig_assignee_individual_name_last']
col_pg_assignee = ['pgpub_id','disambig_assignee_organization',
                   'disambig_assignee_individual_name_first', 'disambig_assignee_individual_name_last']
'''

"\ncol_g_assignee  = ['patent_id','disambig_assignee_organization',\n                   'disambig_assignee_individual_name_first', 'disambig_assignee_individual_name_last']\ncol_pg_assignee = ['pgpub_id','disambig_assignee_organization',\n                   'disambig_assignee_individual_name_first', 'disambig_assignee_individual_name_last']\n"

In [23]:
# set seperator for the data files; for tsv file, use '/t'
seperator = '\t'

In [24]:
# # open assignee data for patents and PGPubs as pandas dataframes
# # - identify the header as row 0 (the first row)
# # - specify columns we want to read via usecols option
# # - use seperator as specified above
# # - infer compression (default but specify this option anyway)
# # - ensure all columns are read in as strings
# # - option keep_default_na=False ensure we get an empty string and not NaN if a value is missing
# patent_assignee_df = pd.read_csv(os_path.join(PATENTSVIEW_DIR,'g_assignee_disambiguated.tsv.zip'), 
#                                  header=0, 
#                                  usecols = col_g_assignee, 
#                                  sep=seperator, 
#                                  compression='infer', 
#                                  dtype=str, 
#                                  keep_default_na=False)
# pgpub_assignee_df =  pd.read_csv(os_path.join(PATENTSVIEW_DIR,'pg_assignee_disambiguated.tsv.zip'), 
#                                  header=0, 
#                                  usecols = col_pg_assignee,
#                                  sep=seperator, 
#                                  compression='infer', 
#                                  dtype=str, 
#                                  keep_default_na=False)

In [25]:
# # as before, create a string combining all assignee name information
# patent_assignee_df['assignee_combined'] = patent_assignee_df['disambig_assignee_organization'] + ' ' + \
#                                           patent_assignee_df['disambig_assignee_individual_name_first'] + ' ' + \
#                                           patent_assignee_df['disambig_assignee_individual_name_last']
# pgpub_assignee_df['assignee_combined']  = pgpub_assignee_df['disambig_assignee_organization'] + ' ' + \
#                                           pgpub_assignee_df['disambig_assignee_individual_name_first'] + ' ' + \
#                                           pgpub_assignee_df['disambig_assignee_individual_name_last']

In [26]:
# # identify if target universities -- patents 
# patent_assignee_df = id_rutgers_stevens_njit(patent_assignee_df, 
#                                              'assignee_combined', 
#                                              output_col_suffix='asg_patent')

In [27]:
# # identify if target universities -- pgpubs
# pgpub_assignee_df  = id_rutgers_stevens_njit(pgpub_assignee_df, 
#                                              'assignee_combined', 
#                                              output_col_suffix='asg_pgpub')

In [28]:
# # summarize the assignee names associated with the target universities - patents
# for univ in ['rutgers_asg_patent', 'stevens_asg_patent', 'njit_asg_patent','princeton_asg_patent','rowan_asg_patent','seton_asg_patent','NJCU_asg_patent','saintPeters_asg_patent']:
#     print('Assignee names for {}'.format(univ))
#     print('----------------------')
#     df = patent_assignee_df.loc[patent_assignee_df[univ]==True]
#     print(df['disambig_assignee_organization'].value_counts())
#     print()

In [29]:
# # summarize the assignee names associated with the target universities - pgpubs
# for univ in ['rutgers_asg_pgpub', 'stevens_asg_pgpub', 'njit_asg_pgpub','princeton_asg_pgpub','rowan_asg_pgpub','NJCU_asg_pgpub','seton_asg_pgpub','saintPeters_asg_pgpub']:
#     print('Assignee names for {}'.format(univ))
#     print('----------------------')
#     df = pgpub_assignee_df.loc[pgpub_assignee_df[univ]==True]
#     print(df['disambig_assignee_organization'].value_counts())
#     print()

In [30]:
# # keep only observations w/target universities - patents
# print('PATENTS')
# print('Number of rows before removing non-target universities: {:,}'.format(len(patent_assignee_df)))

# patent_assignee_df = patent_assignee_df.loc[(patent_assignee_df['rutgers_asg_patent']==True) | \
#                                             (patent_assignee_df['stevens_asg_patent']==True) | \
#                                             (patent_assignee_df['njit_asg_patent']==True)    | \
#                                             (patent_assignee_df['princeton_asg_patent']==True) | \
#                                             (patent_assignee_df['rowan_asg_patent']==True) | \
#                                             (patent_assignee_df['NJCU_asg_patent']==True) | \
#                                             (patent_assignee_df['seton_asg_patent']==True) | \
#                                             (patent_assignee_df['saintPeters_asg_patent']==True)]
# print('Number of rows after removing non-target universities: {:,}'.format(len(patent_assignee_df)))

In [31]:
# # keep only observations w/target universities - pgpubs
# print('PGPUBS')
# print('Number of rows before removing non-target universities: {:,}'.format(len(pgpub_assignee_df)))

# pgpub_assignee_df = pgpub_assignee_df.loc[(pgpub_assignee_df['rutgers_asg_pgpub']==True) | \
#                                           (pgpub_assignee_df['stevens_asg_pgpub']==True) | \
#                                           (pgpub_assignee_df['njit_asg_pgpub']==True)    | \
#                                           (pgpub_assignee_df['rowan_asg_pgpub']==True)    | \
#                                           (pgpub_assignee_df['princeton_asg_pgpub']==True)    | \
#                                           (pgpub_assignee_df['NJCU_asg_pgpub']==True)    | \
#                                           (pgpub_assignee_df['seton_asg_pgpub']==True)    | \
#                                           (pgpub_assignee_df['saintPeters_asg_pgpub']==True)]

# print('Number of rows after removing non-target universities: {:,}'.format(len(pgpub_assignee_df)))

In [32]:
# # keep only the columns we need 
# patent_assignee_df = patent_assignee_df[['patent_id','rutgers_asg_patent','stevens_asg_patent','njit_asg_patent','princeton_asg_patent','rowan_asg_patent','NJCU_asg_patent','seton_asg_patent','saintPeters_asg_patent']]
# pgpub_assignee_df = pgpub_assignee_df[['pgpub_id','rutgers_asg_pgpub','stevens_asg_pgpub','njit_asg_pgpub','rowan_asg_pgpub','princeton_asg_pgpub','NJCU_asg_pgpub','seton_asg_pgpub','saintPeters_asg_pgpub']]

In [33]:
# # create a consoldiated university variable - patents
# patent_assignee_df.loc[patent_assignee_df['rutgers_asg_patent']==True,'univ_asg_patent'] = 'Rutgers'
# patent_assignee_df.loc[patent_assignee_df['stevens_asg_patent']==True,'univ_asg_patent'] = 'Stevens'
# patent_assignee_df.loc[patent_assignee_df['njit_asg_patent']==True,'univ_asg_patent'] = 'NJIT'
# patent_assignee_df.loc[patent_assignee_df['princeton_asg_patent']==True,'univ_asg_patent'] = 'Princeton'
# patent_assignee_df.loc[patent_assignee_df['rowan_asg_patent']==True,'univ_asg_patent'] = 'Rowan'
# patent_assignee_df.loc[patent_assignee_df['NJCU_asg_patent']==True,'univ_asg_patent'] = 'NJCU'
# patent_assignee_df.loc[patent_assignee_df['seton_asg_patent']==True,'univ_asg_patent'] = 'Seton'
# patent_assignee_df.loc[patent_assignee_df['saintPeters_asg_patent']==True,'univ_asg_patent'] = 'SaintPeters'

# # print results 
# print('PATENTS')
# print(patent_assignee_df['univ_asg_patent'].value_counts())

In [34]:
# # create a consoldiated university variable - pgpubs
# pgpub_assignee_df.loc[pgpub_assignee_df['rutgers_asg_pgpub']==True,'univ_asg_pgpub'] = 'Rutgers'
# pgpub_assignee_df.loc[pgpub_assignee_df['stevens_asg_pgpub']==True,'univ_asg_pgpub'] = 'Stevens'
# pgpub_assignee_df.loc[pgpub_assignee_df['njit_asg_pgpub']==True,'univ_asg_pgpub'] = 'NJIT'
# pgpub_assignee_df.loc[pgpub_assignee_df['princeton_asg_pgpub']==True,'univ_asg_pgpub'] = 'Princeton'
# pgpub_assignee_df.loc[pgpub_assignee_df['rowan_asg_pgpub']==True,'univ_asg_pgpub'] = 'Rowan'
# pgpub_assignee_df.loc[pgpub_assignee_df['NJCU_asg_pgpub']==True,'univ_asg_pgpub'] = 'NJCU'
# pgpub_assignee_df.loc[pgpub_assignee_df['seton_asg_pgpub']==True,'univ_asg_pgpub'] = 'Seton'
# pgpub_assignee_df.loc[pgpub_assignee_df['saintPeters_asg_pgpub']==True,'univ_asg_pgpub'] = 'SaintPeters'

# # print results 
# print('PGPUBS')
# print(pgpub_assignee_df['univ_asg_pgpub'].value_counts())

In [35]:
# # define columns we want to use (see above)
# col_g_appln = ['patent_id','application_id']
# col_pg_appln = ['pgpub_id','application_id']

In [36]:
# set separator for .tsv file
seperator = '\t'

In [37]:
# # open data for patents and PGPubs as pandas dataframes
# # - identify the header as row 0 (the first row)
# # - specify columns we want to read via usecols option
# # - use same seperator as above
# # - infer compression (default but specify this option anyway)
# # - ensure all columns are read in as strings
# # - option keep_default_na=False ensure we get an empty string and not NaN if a value is missing
# patent_appln_df = pd.read_csv(os_path.join(PATENTSVIEW_DIR,'g_application.tsv.zip'), 
#                                  header=0, 
#                                  usecols = col_g_appln, 
#                                  sep=seperator, 
#                                  compression='infer', 
#                                  dtype=str, 
#                                  keep_default_na=False)
# pgpub_appln_df =  pd.read_csv(os_path.join(PATENTSVIEW_DIR,'pg_published_application.tsv.zip'), 
#                                  header=0, 
#                                  usecols = col_pg_appln,
#                                  sep=seperator, 
#                                  compression='infer', 
#                                  dtype=str, 
#                                  keep_default_na=False)

In [38]:
# # left join assignee dataframe (left table) to application dataframe (right table) -- patents 
# # use indicator=True to filter results 
# patent_assignee_df = patent_assignee_df.merge(patent_appln_df, 
#                                               how='left', 
#                                               on='patent_id', 
#                                               indicator=True)
# # print merge results 
# patent_assignee_df['_merge'].value_counts()

In [39]:
# # keep _merge=='both'
# patent_assignee_df = patent_assignee_df.loc[patent_assignee_df['_merge']=='both']
# # drop _merge column
# patent_assignee_df.drop(columns=['_merge'], inplace=True)

In [40]:
# # left join assignee dataframe (left table) to application dataframe (right table) -- pgpubs 
# # use indicator=True to filter results 
# pgpub_assignee_df = pgpub_assignee_df.merge(pgpub_appln_df, 
#                                             how='inner', 
#                                             on='pgpub_id', 
#                                             indicator=True)
# # print merge results 
# pgpub_assignee_df['_merge'].value_counts()

In [41]:
# # keep _merge=='both'
# pgpub_assignee_df = pgpub_assignee_df.loc[pgpub_assignee_df['_merge']=='both']
# # drop _merge column
# pgpub_assignee_df.drop(columns=['_merge'], inplace=True)

In [42]:
# # print size of assignee dataframes 
# print('Number patents having target universities as assignee: {:,}'.format(len(patent_assignee_df)))
# print('Number PGPubs having target universities as assignee: {:,}'.format(len(pgpub_assignee_df)))

In [43]:
# initialize the dictionary
appl_dict = {}

In [44]:
# process applicant data by looping through each row of the dataframe
# and calling the function add_appl_owner_to_dict 
for i in range(0,len(applicant_df)): 
    appl_dict = add_appl_owner_to_dict(appl_dict, 
                                       applicant_df.iloc[i]['application_number'], 
                                       applicant_df.iloc[i]['univ'])

In [45]:
# # similarly process patent assignee data; note different column names 
# for i in range(0,len(patent_assignee_df)): 
#     appl_dict = add_appl_owner_to_dict(appl_dict, 
#                                        patent_assignee_df.iloc[i]['application_id'], 
#                                        patent_assignee_df.iloc[i]['univ_asg_patent'])

In [46]:
# # similarly process pgpub assignee data; note different column names 
# for i in range(0,len(pgpub_assignee_df)): 
#     appl_dict = add_appl_owner_to_dict(appl_dict, 
#                                        pgpub_assignee_df.iloc[i]['application_id'], 
#                                        pgpub_assignee_df.iloc[i]['univ_asg_pgpub'])

In [47]:
# to faciliate creating a dataframe, create two lists, one for application IDs and another for universities, 
# where the set of university names is converted to a list
appl_list = []
univ_list = []
for key, value in appl_dict.items():
    appl_list.append(key)
    univ_list.append(list(value))
# verify length of the two list are the same 
n_appl_list = len(appl_list)
n_univ_list = len(univ_list)
print('Length of application ID list: {:,}'.format(n_appl_list))
print('Length of university list: {:,}'.format(n_univ_list))
if n_appl_list!=n_univ_list: 
    raise ValueError('# applications do not equal length of university list')

Length of application ID list: 1,291
Length of university list: 1,291


In [48]:
# create dataframe 
nj_univ_df = pd.DataFrame({'application_number': appl_list, 'univ_list': univ_list})

In [49]:
# create column for application series code
nj_univ_df['series'] = nj_univ_df['application_number'].apply(lambda x: x[0:2]) 

In [50]:
# filter data to remove PCT applications -- begins with "PCT" and hence the series from above = "PC"
print('Number of rows before PCT filter: {:,}'.format(len(nj_univ_df)))
nj_univ_df = nj_univ_df.loc[nj_univ_df['series']!='PC']
print('Number of rows after PCT filter: {:,}'.format(len(nj_univ_df)))

Number of rows before PCT filter: 1,291
Number of rows after PCT filter: 1,291


In [51]:
# print series -- value_counts (pandas 1.5) outputs a pandas series whose index are the series codes
nj_univ_df['series'].value_counts().sort_index()

12      1
13     73
14    268
15    228
16    268
17    271
18     32
61     13
62     96
63     41
Name: series, dtype: int64

In [52]:
# print universities -- these will include those with multiple university owners 
nj_univ_df['univ_list'].value_counts().sort_index()

[NJIT]                   99
[Princeton]             419
[Princeton, Rutgers]      2
[Rowan]                  69
[Rutgers]               595
[Rutgers, NJIT]           4
[Seton]                   7
[Stevens]                85
[Stevens, Rutgers]       11
Name: univ_list, dtype: int64

In [53]:
# to illustrate the next step, look at application '14777191' as an example
print('Before pandas explode()')
print(nj_univ_df.loc[nj_univ_df['application_number']=='14777191'].transpose())

Before pandas explode()
                         325
application_number  14777191
univ_list             [NJIT]
series                    14


In [54]:
# use pandas explode() to break apart multiple universities into seperate rows (keeping same application ID)
print('Number rows prior to breaking apart university lists: {:,}'.format(len(nj_univ_df)))
nj_univ_df = nj_univ_df.explode('univ_list')
print('Number rows after breaking apart university lists: {:,}'.format(len(nj_univ_df)))

Number rows prior to breaking apart university lists: 1,291
Number rows after breaking apart university lists: 1,308


In [55]:
print('After pandas explode()')
print(nj_univ_df.loc[nj_univ_df['application_number']=='14777191'].transpose())

After pandas explode()
                         325
application_number  14777191
univ_list               NJIT
series                    14


In [56]:
# examine result universities -- all should be singular 
nj_univ_df['univ_list'].value_counts().sort_index()

NJIT         103
Princeton    421
Rowan         69
Rutgers      612
Seton          7
Stevens       96
Name: univ_list, dtype: int64

In [57]:
# rename univ_list column to univ
nj_univ_df.rename(columns={'univ_list': 'univ'}, inplace=True)

In [58]:
# # delete unneeded dataframes and data structures 
# del applicant_df, patent_assignee_df, pgpub_assignee_df
# del appl_dict, appl_list, univ_list

In [59]:
# look at first 5 observations (do not use transpose to preserve column orientation)
# note that university names are not in a list 
nj_univ_df.head(5)

,application_number,univ,series
0,17435954,Rutgers,17
1,17914647,Rutgers,17
2,18005188,Princeton,18
3,18245256,Rutgers,18
4,12539049,NJIT,12


In [60]:
# look at the first 6 observations (3 applications) having more than one university owner 
# (option keep=False tags all duplicate observation, not just the first or last)
# (note: this command does not change the underlying dataset - duplicates are not dropped)
nj_univ_df.loc[nj_univ_df.duplicated(subset=['application_number'], keep=False)].head(6)

,application_number,univ,series
140,14348197,Stevens,14
140,14348197,Rutgers,14
293,14751086,Stevens,14
293,14751086,Rutgers,14
299,14763944,Rutgers,14
299,14763944,NJIT,14


In [61]:
# set filepath to PatEx application data csv file
filepath = os_path.join(PATEX_DIR, 'application_data.csv')
print(filepath)

C:\Users\user\Downloads\BulkData2023\application_data.csv


In [62]:
# identify columns we want to use
col_list = ['application_number','filing_date','application_invention_type',
            'earliest_pgpub_number','earliest_pgpub_date','patent_number','patent_issue_date',
            'invention_title','small_entity_indicator','aia_first_to_file']

# open csv file as a pandas dataframe
# - identify the header as row 0 (the first row)
# - use only selected columns
# - ensure all columns are read in as strings (important for the application number)
# - option keep_default_na=False ensure we get an empty string and not NaN if a value is missing
appl_df = pd.read_csv(filepath,
                      header=0,
                      usecols=col_list,
                      dtype=str,
                      keep_default_na=False)

In [63]:
appl_df.head()

,application_number,filing_date,application_invention_type,earliest_pgpub_number,earliest_pgpub_date,patent_number,patent_issue_date,invention_title,small_entity_indicator,aia_first_to_file
0,04453098,,,,,,,,UNDISCOUNTED,
1,04544040,,,,,,,,UNDISCOUNTED,
2,04577552,,,,,,,,UNDISCOUNTED,
3,04589670,,,,,,,,UNDISCOUNTED,
4,04578318,,,,,,,,UNDISCOUNTED,


In [64]:
# create column for filing year
appl_df['filing_year'] = appl_df['filing_date'].apply(lambda x: x[0:4])

In [65]:
appl_df.head()

,application_number,filing_date,application_invention_type,earliest_pgpub_number,earliest_pgpub_date,patent_number,patent_issue_date,invention_title,small_entity_indicator,aia_first_to_file,filing_year
0,04453098,,,,,,,,UNDISCOUNTED,,
1,04544040,,,,,,,,UNDISCOUNTED,,
2,04577552,,,,,,,,UNDISCOUNTED,,
3,04589670,,,,,,,,UNDISCOUNTED,,
4,04578318,,,,,,,,UNDISCOUNTED,,


In [66]:
# filter data to keep applications between start and end dates

print('Number of rows before date filter: {:,}'.format(len(appl_df)))

appl_df = appl_df.loc[(appl_df['filing_year']>=START_YEAR) & \
                      (appl_df['filing_year']<=END_YEAR)]

print('Number of rows after date filter: {:,}'.format(len(appl_df)))


Number of rows before date filter: 14,100,378
Number of rows after date filter: 4,246,035


In [67]:
# left join nj_univ_df (left table) to appl_df (right table); use indicator=True to filter results 
nj_univ_df = nj_univ_df.merge(appl_df, 
                                how='left', 
                                on='application_number', 
                                indicator=True)

In [68]:
nj_univ_df.head()

,application_number,univ,series,filing_date,application_invention_type,earliest_pgpub_number,earliest_pgpub_date,patent_number,patent_issue_date,invention_title,small_entity_indicator,aia_first_to_file,filing_year,_merge
0,17435954,Rutgers,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1,17914647,Rutgers,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2,18005188,Princeton,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3,18245256,Rutgers,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4,12539049,NJIT,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [69]:
merge_results_series = nj_univ_df['_merge'].value_counts()
print(merge_results_series)

print('\nMerge results: ')
print('Number appl w/target NJ university applicants within date range: {:,}'.
      format(merge_results_series['both']))
print('Number appl w/target NJ university applicants outside date range (or no appl data): {:,}'.
      format(merge_results_series['left_only']))

both          1053
left_only      255
right_only       0
Name: _merge, dtype: int64

Merge results: 
Number appl w/target NJ university applicants within date range: 1,053
Number appl w/target NJ university applicants outside date range (or no appl data): 255


In [70]:
# keep _merge=='both'
nj_univ_df = nj_univ_df.loc[nj_univ_df['_merge']=='both']

In [71]:
# drop _merge column
nj_univ_df.drop(columns=['_merge'], inplace=True)

In [72]:
# reset index; do not insert previous index into dataframe
nj_univ_df.reset_index(drop=True, inplace=True)

In [73]:
# delete application dataframe 
del appl_df 

In [74]:
# print first five rows of data (use transpose for readability)
nj_univ_df.head(5).transpose()

,0,1,2,3,4
application_number,14605053,14417704,14607496,14418392,14609269
univ,Stevens,Rowan,Rutgers,Rutgers,Rutgers
series,14,14,14,14,14
filing_date,2015-01-26,2015-01-27,2015-01-28,2015-01-29,2015-01-29
application_invention_type,Utility,Utility,Utility,Utility,Utility
earliest_pgpub_number,US20150235073A1,US20150198600A1,US20150233934A1,US20150297725A1,US20150140074A1
earliest_pgpub_date,2015-08-20,2015-07-16,2015-08-20,2015-10-22,2015-05-21
patent_number,,,9945051,,9775907
patent_issue_date,,,2018-04-17,,2017-10-03
invention_title,FLEXIBLE PART-BASED REPRESENTATION FOR REAL-WO...,Autoantibody Profiles in the Early Detection a...,COLOR-ENCODING AND IN-SITU INTERROGATION OF MA...,COCHLEATES MADE WITH SOY PHOSPHATIDYLSERINE,COCHLEATES MADE WITH SOY PHOSPHATIDYLSERINE


In [75]:
# set filepath to PatEx inventor data csv file
filepath = os_path.join(PATEX_DIR, 'all_inventors.csv')
print(filepath)

C:\Users\user\Downloads\BulkData2023\all_inventors.csv


In [76]:
# open csv file as a pandas dataframe
# - identify the header as row 0 (the first row)
# - use all columns (do not specify usecols)
# - ensure all columns are read in as strings (important for the application number)
# - option keep_default_na=False ensure we get an empty string and not NaN if a value is missing
inv_df = pd.read_csv(filepath,
                     header=0,
                     dtype=str,
                     keep_default_na=False)

In [77]:
# left join applicant_df (left table) to inv_df (right table); use indicator=True to check results 
nj_univ_appl_inv_df = nj_univ_df.merge(inv_df, 
                                       how='left', 
                                       on='application_number', 
                                       indicator=True)

In [78]:
merge_results_series = nj_univ_appl_inv_df['_merge'].value_counts()
print(merge_results_series)

print('\nMerge results: ')
print('Number application-applicant observations with inventor data: {:,}'.
      format(merge_results_series['both']))
print('Number application-applicant observations missing inventor data: {:,}'.
      format(merge_results_series['left_only']))

both          3457
left_only        9
right_only       0
Name: _merge, dtype: int64

Merge results: 
Number application-applicant observations with inventor data: 3,457
Number application-applicant observations missing inventor data: 9


In [79]:
# keep _merge=='both'
nj_univ_appl_inv_df = nj_univ_appl_inv_df.loc[nj_univ_appl_inv_df['_merge']=='both']

In [80]:
# drop _merge column 
nj_univ_appl_inv_df.drop(columns=['_merge'], inplace=True)

In [81]:
# delete unneeded dataframes
del inv_df, nj_univ_df

In [82]:
nj_univ_appl_inv_df['inv_id'] = nj_univ_appl_inv_df.groupby(['inventor_name_last',
                                                             'inventor_name_first',
                                                             'inventor_name_middle']).ngroup()

In [83]:
# count number of unque inventors by university
nj_univ_appl_inv_df.groupby(by=['univ'])['inv_id'].nunique()

univ
NJIT         175
Princeton    777
Rowan        138
Rutgers      938
Seton         11
Stevens      147
Name: inv_id, dtype: int64

In [84]:
# import gender_it_functions
from gender_it import gender_it_functions as gf

In [85]:
#gf.read_wgnd(path='gender_it//')

In [86]:
# get gender attributions for inventors; set threshold to 0.60
nj_univ_appl_inv_gender_df = gf.get_gender(nj_univ_appl_inv_df, 
                                           name_column = 'inventor_name_first', 
                                           country_column = 'inventor_country_code', 
                                           treshold = 0.6)

Step 1 - reading the name-country-gender dictionary
reading the dictionnary.
Step 2 - reading the name-language-gender dictionary
reading the dictionnary.
downloading the dictionnary.
Step 3 - reading the name-gender dictionary.
reading the dictionnary.
Results distirbution is as follow: 
            gender  Percentage
M            2409   69.684698
F             658   19.033844
not found     388   11.223604
?               2    0.057854


In [87]:
nj_univ_appl_inv_df.head()

,application_number,univ,series,filing_date,application_invention_type,earliest_pgpub_number,earliest_pgpub_date,patent_number,patent_issue_date,invention_title,...,aia_first_to_file,filing_year,inventor_name_first,inventor_name_middle,inventor_name_last,inventor_rank,inventor_city_name,inventor_region_code,inventor_country_code,inv_id
0,14605053,Stevens,14,2015-01-26,Utility,US20150235073A1,2015-08-20,,,FLEXIBLE PART-BASED REPRESENTATION FOR REAL-WO...,...,true,2015,Gang,,Hua,1,Livingston,NJ,US,794
1,14605053,Stevens,14,2015-01-26,Utility,US20150235073A1,2015-08-20,,,FLEXIBLE PART-BASED REPRESENTATION FOR REAL-WO...,...,true,2015,Haoxiang,,Li,2,Hoboken,NJ,US,1052
2,14417704,Rowan,14,2015-01-27,Utility,US20150198600A1,2015-07-16,,,Autoantibody Profiles in the Early Detection a...,...,false,2015,Robert,G.,Nagele,1,Turnersville,NJ,US,1297
3,14417704,Rowan,14,2015-01-27,Utility,US20150198600A1,2015-07-16,,,Autoantibody Profiles in the Early Detection a...,...,false,2015,Eric,P.,Nagele,2,Turnersville,NJ,US,1296
4,14607496,Rutgers,14,2015-01-28,Utility,US20150233934A1,2015-08-20,9945051,2018-04-17,COLOR-ENCODING AND IN-SITU INTERROGATION OF MA...,...,false,2015,Richard,,Ebright,1,New Brunswick,NJ,US,486


In [88]:
nj_univ_appl_inv_gender_df.head()

,application_number,univ,series,filing_date,application_invention_type,earliest_pgpub_number,earliest_pgpub_date,patent_number,patent_issue_date,invention_title,...,inventor_rank,inventor_city_name,inventor_region_code,inventor_country_code,inv_id,level,gender,F,M,?
786,14605053,Stevens,14,2015-01-26,Utility,US20150235073A1,2015-08-20,,,FLEXIBLE PART-BASED REPRESENTATION FOR REAL-WO...,...,1,Livingston,NJ,US,794,1.0,M,0.0,1.0,0.0
388,14605053,Stevens,14,2015-01-26,Utility,US20150235073A1,2015-08-20,,,FLEXIBLE PART-BASED REPRESENTATION FOR REAL-WO...,...,2,Hoboken,NJ,US,1052,3.0,M,0.0,1.0,0.0
3039,14417704,Rowan,14,2015-01-27,Utility,US20150198600A1,2015-07-16,,,Autoantibody Profiles in the Early Detection a...,...,1,Turnersville,NJ,US,1297,2.0,M,0.0,1.0,0.0
2174,14417704,Rowan,14,2015-01-27,Utility,US20150198600A1,2015-07-16,,,Autoantibody Profiles in the Early Detection a...,...,2,Turnersville,NJ,US,1296,2.0,M,0.0,1.0,0.0
2993,14607496,Rutgers,14,2015-01-28,Utility,US20150233934A1,2015-08-20,9945051,2018-04-17,COLOR-ENCODING AND IN-SITU INTERROGATION OF MA...,...,1,New Brunswick,NJ,US,486,2.0,M,0.0,1.0,0.0


In [89]:
nj_univ_appl_inv_gender_df.columns

Index(['application_number', 'univ', 'series', 'filing_date',
       'application_invention_type', 'earliest_pgpub_number',
       'earliest_pgpub_date', 'patent_number', 'patent_issue_date',
       'invention_title', 'small_entity_indicator', 'aia_first_to_file',
       'filing_year', 'inventor_name_first', 'inventor_name_middle',
       'inventor_name_last', 'inventor_rank', 'inventor_city_name',
       'inventor_region_code', 'inventor_country_code', 'inv_id', 'level',
       'gender', 'F', 'M', '?'],
      dtype='object')

In [90]:
nj_univ_appl_inv_gender_df.groupby(by=["univ","gender","filing_year"]).size()

univ     gender     filing_year
NJIT     F          2015           10
                    2016            6
                    2017            4
                    2018            5
                    2019            8
                                   ..
Stevens  not found  2017            1
                    2019            5
                    2020            4
                    2021            3
                    2022           10
Length: 124, dtype: int64

In [91]:
nj_univ_appl_inv_gender_df.groupby(by=["univ","gender","filing_year"])['inv_id'].nunique()

univ     gender     filing_year
NJIT     F          2015           9
                    2016           6
                    2017           4
                    2018           5
                    2019           8
                                  ..
Stevens  not found  2017           1
                    2019           5
                    2020           4
                    2021           3
                    2022           9
Name: inv_id, Length: 124, dtype: int64

In [92]:
nj_univ_appl_inv_gender_df.groupby(by=["univ","application_invention_type","filing_year","gender"])['inv_id'].nunique()

univ     application_invention_type  filing_year  gender   
NJIT     Provisional                 2016         F             1
                                                  M             5
                                                  not found     2
                                     2017         M             4
                                     2018         F             1
                                                               ..
Stevens  Utility                     2021         F             7
                                                  M            13
                                     2022         F             4
                                                  M            24
                                                  not found     9
Name: inv_id, Length: 204, dtype: int64

In [93]:
nj_univ_appl_inv_gender_df.groupby(by=["univ","application_invention_type","filing_year","gender"])['application_number'].nunique()

univ     application_invention_type  filing_year  gender   
NJIT     Provisional                 2016         F             1
                                                  M             3
                                                  not found     2
                                     2017         M             2
                                     2018         F             1
                                                               ..
Stevens  Utility                     2021         F             4
                                                  M             4
                                     2022         F             5
                                                  M            11
                                                  not found     8
Name: application_number, Length: 204, dtype: int64

In [94]:
nj_univ_appl_inv_gender_df.to_csv('first_results_with_asg.csv')